In [ ]:
!unzip -q /content/drive/MyDrive/projet_advanced/dataset_subtype.zip  -d /content/dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X_train=pd.read_csv('/content/dataset/X_train.csv')
X_test=pd.read_csv('/content/dataset/X_test.csv')
X_val=pd.read_csv('/content/dataset/X_val.csv')
y_train=pd.read_csv('/content/dataset/y_train.csv')
y_test=pd.read_csv('/content/dataset/y_test.csv')
y_val=pd.read_csv('/content/dataset/y_val.csv')

### **Encodage du label (obligatoire)**

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train.values.ravel())
y_val   = label_encoder.transform(y_val.values.ravel())
y_test  = label_encoder.transform(y_test.values.ravel())

# Vérification
print("Classes :", label_encoder.classes_)


Classes : ['Audio' 'Background' 'Bruteforce DNS' 'Bruteforce FTP' 'Bruteforce HTTP'
 'Bruteforce SSH' 'Bruteforce Telnet' 'DoS ACK' 'DoS CWR' 'DoS ECN'
 'DoS FIN' 'DoS HTTP' 'DoS ICMP' 'DoS MAC' 'DoS PSH' 'DoS RST' 'DoS SYN'
 'DoS UDP' 'DoS URG' 'Information Gathering' 'Mirai DDoS ACK'
 'Mirai DDoS DNS' 'Mirai DDoS GREETH' 'Mirai DDoS GREIP' 'Mirai DDoS HTTP'
 'Mirai DDoS SYN' 'Mirai DDoS UDP' 'Mirai Scan Bruteforce' 'Text'
 'Video HTTP' 'Video RTP' 'Video UDP']


### **Normalisation des features (TRÈS IMPORTANT)**

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 1️⃣ Fit uniquement sur le train
scaler.partial_fit(X_train)

# 2️⃣ Fonction de scaling par batch
def scale_in_batches(X, scaler, batch_size=50_000):
    X = X.values if hasattr(X, "values") else X
    X_scaled = np.empty(X.shape, dtype=np.float32)

    for i in range(0, X.shape[0], batch_size):
        X_scaled[i:i+batch_size] = scaler.transform(X[i:i+batch_size])

    return X_scaled

# 3️⃣ Scaling sécurisé
X_train_scaled = scale_in_batches(X_train, scaler)
X_val_scaled   = scale_in_batches(X_val, scaler)
X_test_scaled  = scale_in_batches(X_test, scaler)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

In [ ]:
import torch
del X_train, X_val, X_test
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
print("Train :", X_train_scaled.shape, y_train.shape)
print("Val   :", X_val_scaled.shape, y_val.shape)
print("Test  :", X_test_scaled.shape, y_test.shape)
print("Labels uniques :", np.unique(y_train))


Train : (5076507, 77) (5076507,)
Val   : (1087824, 77) (1087824,)
Test  : (1087824, 77) (1087824,)
Labels uniques : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]


### **L’équilibrage se fait UNIQUEMENT sur le TRAIN**

In [ ]:
import pandas as pd

# IMPORTANT : utiliser X_train_scaled
train_df = pd.concat([
    pd.DataFrame(X_train_scaled),
    pd.Series(y_train, name="label").reset_index(drop=True)
], axis=1)

label_col = "label"

print(train_df[label_col].value_counts())


label
15    681344
7     550006
14    543288
19    519985
16    510660
18    504713
9     495528
8     493886
10    457470
17    179513
11     54177
21     38602
2      15034
25      9748
24      5753
27      4109
6       3409
5       2749
20      2610
3       2439
4        440
29       254
30       245
28       147
0        133
31       101
26        49
23        35
22        30
1         22
13        21
12         7
Name: count, dtype: int64


In [ ]:
M = 125000
print("Taille cible par classe :", M)


Taille cible par classe : 125000


In [ ]:
from sklearn.utils import resample

balanced_dfs = []

classes = train_df[label_col].unique()
print("Classes :", classes)

for cls in classes:
    df_cls = train_df[train_df[label_col] == cls]
    n_cls = len(df_cls)

    print(f"Classe {cls} : {n_cls} échantillons")

    if n_cls > M:
        # Down-sampling
        df_resampled = resample(
            df_cls,
            replace=False,
            n_samples=M,
            random_state=42
        )
    else:
        # Up-sampling
        df_resampled = resample(
            df_cls,
            replace=True,
            n_samples=M,
            random_state=42
        )

    balanced_dfs.append(df_resampled)


Classes : [18 19  9 14 17 16 15  7  8 10 25 11 21  2 24  6 27 20  5  1  3 28 29  4
 30 26  0 23 31 13 22 12]
Classe 18 : 504713 échantillons
Classe 19 : 519985 échantillons
Classe 9 : 495528 échantillons
Classe 14 : 543288 échantillons
Classe 17 : 179513 échantillons
Classe 16 : 510660 échantillons
Classe 15 : 681344 échantillons
Classe 7 : 550006 échantillons
Classe 8 : 493886 échantillons
Classe 10 : 457470 échantillons
Classe 25 : 9748 échantillons
Classe 11 : 54177 échantillons
Classe 21 : 38602 échantillons
Classe 2 : 15034 échantillons
Classe 24 : 5753 échantillons
Classe 6 : 3409 échantillons
Classe 27 : 4109 échantillons
Classe 20 : 2610 échantillons
Classe 5 : 2749 échantillons
Classe 1 : 22 échantillons
Classe 3 : 2439 échantillons
Classe 28 : 147 échantillons
Classe 29 : 254 échantillons
Classe 4 : 440 échantillons
Classe 30 : 245 échantillons
Classe 26 : 49 échantillons
Classe 0 : 133 échantillons
Classe 23 : 35 échantillons
Classe 31 : 101 échantillons
Classe 13 : 21 échan

In [ ]:
train_balanced_df = pd.concat(balanced_dfs) \
    .sample(frac=1, random_state=42) \
    .reset_index(drop=True)


In [ ]:
del train_df, balanced_dfs
gc.collect()
torch.cuda.empty_cache()


In [ ]:
X_train_balanced = train_balanced_df.iloc[:, :-1].values
y_train_balanced = train_balanced_df.iloc[:, -1].values


In [ ]:
del train_balanced_df
gc.collect()
torch.cuda.empty_cache()


In [ ]:
pd.Series(y_train_balanced).value_counts()


,count
8,125000
19,125000
20,125000
10,125000
3,125000
2,125000
5,125000
24,125000
27,125000
1,125000


### **One hot encoding**

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# reshape nécessaire car y_train_balanced est 1D
y_train_balanced = y_train_balanced.reshape(-1, 1)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Pour les versions récentes de scikit-learn
ohe = OneHotEncoder(sparse_output=False)
# Fit et transform
y_train_oh = ohe.fit_transform(y_train_balanced)

# Vérification
print("Shape One-Hot :", y_train_oh.shape)
print("Classes :", ohe.categories_)


Shape One-Hot : (4000000, 32)
Classes : [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])]


In [ ]:
y_val_oh = ohe.transform(y_val.reshape(-1, 1))
y_test_oh = ohe.transform(y_test.reshape(-1, 1))


### **Implémentation FWM**

In [ ]:
import numpy as np

def compute_fwm_weights_multiclass_stable(X, y, eps=1e-8):
    """
    Calcul des poids FWM pour un dataset multiclass (32 classes ou plus)
    avec correction pour classes de petite taille.

    X : ndarray (n_samples, n_features)
    y : ndarray (n_samples,) labels entiers 0..C-1
    eps : float pour éviter division par 0

    return :
        weights : ndarray (n_features,)
    """
    classes, counts = np.unique(y, return_counts=True)
    n_features = X.shape[1]

    mu_total = np.mean(X, axis=0)
    scores = np.zeros(n_features, dtype=np.float32)

    for cls, count in zip(classes, counts):
        X_cls = X[y == cls]
        mu_cls = np.mean(X_cls, axis=0)
        std_cls = np.std(X_cls, axis=0)

        # pondération par taille de classe + eps pour stabilité
        weight_factor = count / np.sum(counts)
        scores += weight_factor * np.abs(mu_cls - mu_total) / (std_cls + eps)

    # Normalisation douce [0,1]
    weights = scores / (np.max(scores) + eps)

    return weights.astype(np.float32)


In [ ]:
def apply_fwm_in_batches(X, fwm_weights, alpha=0.2, batch_size=500000):
    """
    Application FWM par batch avec alpha = intensité
    X : ndarray (n_samples, n_features)
    fwm_weights : ndarray (n_features,)
    """
    n_samples = X.shape[0]
    X_weighted = np.empty_like(X, dtype=np.float32)

    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch = X[start:end]
        # Application douce : on ne détruit pas l'information originale
        X_weighted[start:end] = batch * (1.0 + alpha * fwm_weights)

    return X_weighted


In [ ]:
# 1. Calcul des poids (train ONLY)
y_train_balanced_1d = y_train_balanced.reshape(-1)
fwm_weights = compute_fwm_weights_multiclass_stable(
    X_train_balanced,
    y_train_balanced_1d
)

# 2. Application FWM SUR LES MÊMES TYPES DE DONNÉES
X_train_fwm = apply_fwm_in_batches(X_train_balanced, fwm_weights,alpha=0.2)
X_val_fwm   = apply_fwm_in_batches(X_val_scaled,   fwm_weights,alpha=0.2)
X_test_fwm  = apply_fwm_in_batches(X_test_scaled,  fwm_weights,alpha=0.2)


In [ ]:
X_train_fwm = X_train_fwm[:, None, :]
X_val_fwm   = X_val_fwm[:, None, :]
X_test_fwm  = X_test_fwm[:, None, :]


### **implémentattion du swcc**

In [ ]:
import numpy as np
from tqdm import tqdm

def swcc_single_batch(X_batch, window_size=5):
    """
    Applique SWCC sur un batch de données.
    X_batch: ndarray de forme (n_samples, n_features)
    window_size: taille de la fenêtre glissante
    """
    X_corrected = np.empty_like(X_batch, dtype=np.float32)
    n_samples, n_features = X_batch.shape

    for i in range(n_features):
        channel = X_batch[:, i]
        corrected_channel = np.zeros_like(channel, dtype=np.float32)

        for j in range(n_samples):
            start = max(0, j - window_size // 2)
            end = min(n_samples, j + window_size // 2 + 1)
            window = channel[start:end]

            corrected_channel[j] = channel[j] - np.mean(window)

        X_corrected[:, i] = corrected_channel

    return X_corrected


def apply_swcc_in_batches(X, batch_size=500000, window_size=5):
    """
    Applique SWCC sur l'ensemble des données par batch.
    """
    n_samples = X.shape[0]
    X_corrected = np.empty_like(X, dtype=np.float32)

    for start in tqdm(range(0, n_samples, batch_size), desc="SWCC"):
        end = min(start + batch_size, n_samples)
        X_corrected[start:end] = swcc_single_batch(X[start:end], window_size=window_size)

    return X_corrected


In [ ]:
# Train équilibré
X_train_swcc = apply_swcc_in_batches(X_train_balanced, batch_size=500000, window_size=5)

# Val et Test (non équilibré, juste scaled)
X_val_swcc   = apply_swcc_in_batches(X_val_scaled, batch_size=200000, window_size=5)
X_test_swcc  = apply_swcc_in_batches(X_test_scaled, batch_size=200000, window_size=5)
print("Shapes SWCC :")
print("Train :", X_train_swcc.shape)
print("Val   :", X_val_swcc.shape)
print("Test  :", X_test_swcc.shape)

SWCC: 100%|██████████| 6/6 [16:00<00:00, 160.05s/it]

Shapes SWCC :
Train : (4000000, 77)
Val   : (1087824, 77)
Test  : (1087824, 77)


### **Modelisation CNN**

In [ ]:
import torch
import torch.nn as nn

class SimpleCNN_OneHot(nn.Module):
    def __init__(self, input_channels=1, num_classes=32):
        super(SimpleCNN_OneHot, self).__init__()

        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(16)

        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(32)

        self.relu  = nn.ReLU()
        self.pool  = nn.AdaptiveAvgPool1d(1)

        self.fc    = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)          # logits (PAS de sigmoid ici)
        return x


In [ ]:
def train_cnn_multiclass_ohe(
    model,
    X_train, y_train,
    X_val, y_val,
    epochs=20,
    batch_size=256,
    lr=0.001,
    pos_weight=None  # <- optionnel (fortement recommandé si déséquilibré)
):
    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader, TensorDataset
    import numpy as np

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ---------- Prepare X ----------
    def prepare_X(X):
        X = torch.tensor(X, dtype=torch.float32)
        if X.ndim == 2:
            X = X.unsqueeze(1)  # (B, 1, L)
        return X

    X_train_t = prepare_X(X_train)
    X_val_t   = prepare_X(X_val)

    # ---------- Prepare y (ONE-HOT FLOAT) ----------
    y_train_t = torch.tensor(y_train, dtype=torch.float32)
    y_val_t   = torch.tensor(y_val, dtype=torch.float32)

    assert y_train_t.ndim == 2, "y_train doit être One-Hot (B, C)"
    assert y_val_t.ndim == 2, "y_val doit être One-Hot (B, C)"

    train_loader = DataLoader(
        TensorDataset(X_train_t, y_train_t),
        batch_size=batch_size,
        shuffle=True
    )

    val_loader = DataLoader(
        TensorDataset(X_val_t, y_val_t),
        batch_size=batch_size,
        shuffle=False
    )

    # ---------- Loss ----------
    if pos_weight is not None:
        pos_weight = torch.tensor(pos_weight, dtype=torch.float32).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    else:
        criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_acc = 0.0
    best_state = model.state_dict()

    for epoch in range(epochs):
        # ===== TRAIN =====
        model.train()
        correct, total = 0, 0

        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(Xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            preds = torch.argmax(logits, dim=1)
            targets = torch.argmax(yb, dim=1)

            correct += (preds == targets).sum().item()
            total += yb.size(0)

        train_acc = correct / total

        # ===== VALIDATION =====
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)

                logits = model(Xb)
                preds = torch.argmax(logits, dim=1)
                targets = torch.argmax(yb, dim=1)

                correct += (preds == targets).sum().item()
                total += yb.size(0)

        val_acc = correct / total

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train Acc: {train_acc:.4f} | "
            f"Val Acc: {val_acc:.4f}"
        )

    model.load_state_dict(best_state)
    return model


### **CNN-1 : sur données normales**

In [ ]:
model = SimpleCNN_OneHot(input_channels=1, num_classes=32)
cnn1_model = train_cnn_multiclass_ohe(
    model,
    X_train=X_train_balanced,
    y_train=y_train_oh,   # (N, 8) float
    X_val=X_val_scaled,
    y_val=y_val_oh,       # (N, 8) float
    epochs=20,
    batch_size=256,
    lr=0.001
)

Epoch 1/20 | Train Acc: 0.4141 | Val Acc: 0.2314
Epoch 2/20 | Train Acc: 0.6157 | Val Acc: 0.2799
Epoch 3/20 | Train Acc: 0.6753 | Val Acc: 0.3394
Epoch 4/20 | Train Acc: 0.7140 | Val Acc: 0.2564
Epoch 5/20 | Train Acc: 0.7438 | Val Acc: 0.3954
Epoch 6/20 | Train Acc: 0.7640 | Val Acc: 0.3151
Epoch 7/20 | Train Acc: 0.7779 | Val Acc: 0.2100
Epoch 8/20 | Train Acc: 0.7888 | Val Acc: 0.5961
Epoch 9/20 | Train Acc: 0.7982 | Val Acc: 0.3179
Epoch 10/20 | Train Acc: 0.8060 | Val Acc: 0.6064
Epoch 11/20 | Train Acc: 0.8127 | Val Acc: 0.3970
Epoch 12/20 | Train Acc: 0.8176 | Val Acc: 0.3845
Epoch 13/20 | Train Acc: 0.8224 | Val Acc: 0.5106
Epoch 14/20 | Train Acc: 0.8262 | Val Acc: 0.6083
Epoch 15/20 | Train Acc: 0.8299 | Val Acc: 0.4894
Epoch 16/20 | Train Acc: 0.8331 | Val Acc: 0.4617
Epoch 17/20 | Train Acc: 0.8363 | Val Acc: 0.2744
Epoch 18/20 | Train Acc: 0.8387 | Val Acc: 0.4031
Epoch 19/20 | Train Acc: 0.8409 | Val Acc: 0.5660
Epoch 20/20 | Train Acc: 0.8427 | Val Acc: 0.4742


In [ ]:
def evaluate_model_multiclass_ohe(model, X_test, y_test, batch_size=256):
    import torch
    import numpy as np
    from sklearn.metrics import (
        confusion_matrix,
        accuracy_score,
        f1_score,
        classification_report
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)

    # ---------- X ----------
    if hasattr(X_test, "values"):
        X_test = X_test.values

    X_tensor = torch.tensor(X_test, dtype=torch.float32)
    if X_tensor.ndim == 2:
        X_tensor = X_tensor.unsqueeze(1)

    # ---------- y (One-Hot → class index) ----------
    if hasattr(y_test, "values"):
        y_test = y_test.values

    y_test = np.array(y_test, dtype=np.float32)
    y_true = np.argmax(y_test, axis=1)   # 🔑 OHE → labels int

    y_tensor = torch.tensor(y_test, dtype=torch.float32)

    dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
    loader  = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    all_preds = []

    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            logits = model(Xb)

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())

    all_preds = np.array(all_preds)

    # ---------- Metrics ----------
    acc = accuracy_score(y_true, all_preds)
    f1_macro = f1_score(y_true, all_preds, average="macro")
    f1_weighted = f1_score(y_true, all_preds, average="weighted")
    cm = confusion_matrix(y_true, all_preds)

    print("Accuracy :", round(acc, 4))
    print("F1 macro :", round(f1_macro, 4))
    print("F1 weighted :", round(f1_weighted, 4))
    print("\nConfusion Matrix:")
    print(cm)

    print("\nClassification Report:")
    print(classification_report(y_true, all_preds, digits=4))

    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "confusion_matrix": cm
    }


In [ ]:
results = evaluate_model_multiclass_ohe(
    model=cnn1_model,
    X_test=X_test_scaled,
    y_test=y_test_oh,   # One-Hot
    batch_size=512
)


Accuracy : 0.4747
F1 macro : 0.2894
F1 weighted : 0.4037

Confusion Matrix:
[[  21    0    0 ...    1    0    0]
 [   0    0    0 ...    5    0    0]
 [   0    0  119 ... 1947    0    0]
 ...
 [   0    0    0 ...   37    0    0]
 [   0    0    0 ...   11   14    9]
 [   0    0    0 ...    3    9    3]]

Classification Report:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0     0.9545    0.7241    0.8235        29
           1     0.0000    0.0000    0.0000         5
           2     1.0000    0.0369    0.0713      3221
           3     1.0000    0.9732    0.9864       523
           4     1.0000    0.0957    0.1748        94
           5     0.0000    0.0000    0.0000       589
           6     0.0000    0.0000    0.0000       730
           7     0.8984    0.1710    0.2873    117859
           8     0.3421    0.8998    0.4957    105833
           9     0.9980    0.1080    0.1948    106185
          10     0.2616    0.7103    0.3823     98030
          11     0.9488    0.5040    0.6583     11610
          12     0.0000    0.0000    0.0000         1
          13     1.0000    1.0000    1.0000         4
          14     0.8089    0.5725    0.6705    116419
          15     0.6842    0.0725    0.1311    146003
          16     0.4599    0.8426    0.5950    109427
          17     0.6107    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Chemin complet dans Google Drive
save_path = "/content/drive/MyDrive/projet_advanced/cnn1_model.pth"

# Sauvegarde du modèle (state_dict recommandé)
torch.save(cnn1_model.state_dict(), save_path)

print(f"Modèle sauvegardé dans : {save_path}")


Modèle sauvegardé dans : /content/drive/MyDrive/projet_advanced/cnn1_model.pth


### **FWM-CNN**

In [ ]:
model = SimpleCNN_OneHot(input_channels=1, num_classes=32)
fwm_cnn_model = train_cnn_multiclass_ohe(
    model,
    X_train=X_train_fwm,
    y_train=y_train_oh,
    X_val=X_val_fwm,
    y_val=y_val_oh,
    epochs=20,
    batch_size=256,
    lr=0.001
)

Epoch 1/20 | Train Acc: 0.3484 | Val Acc: 0.2164
Epoch 2/20 | Train Acc: 0.5059 | Val Acc: 0.2699
Epoch 3/20 | Train Acc: 0.5754 | Val Acc: 0.3267
Epoch 4/20 | Train Acc: 0.6238 | Val Acc: 0.2198
Epoch 5/20 | Train Acc: 0.6564 | Val Acc: 0.3011
Epoch 6/20 | Train Acc: 0.6785 | Val Acc: 0.3853
Epoch 7/20 | Train Acc: 0.6977 | Val Acc: 0.2931
Epoch 8/20 | Train Acc: 0.7126 | Val Acc: 0.4676
Epoch 9/20 | Train Acc: 0.7249 | Val Acc: 0.3128
Epoch 10/20 | Train Acc: 0.7346 | Val Acc: 0.3242
Epoch 11/20 | Train Acc: 0.7436 | Val Acc: 0.1975
Epoch 12/20 | Train Acc: 0.7521 | Val Acc: 0.3163
Epoch 13/20 | Train Acc: 0.7594 | Val Acc: 0.3574
Epoch 14/20 | Train Acc: 0.7657 | Val Acc: 0.3230
Epoch 15/20 | Train Acc: 0.7712 | Val Acc: 0.4433
Epoch 16/20 | Train Acc: 0.7766 | Val Acc: 0.4478
Epoch 17/20 | Train Acc: 0.7810 | Val Acc: 0.3032
Epoch 18/20 | Train Acc: 0.7862 | Val Acc: 0.2987
Epoch 19/20 | Train Acc: 0.7894 | Val Acc: 0.4145
Epoch 20/20 | Train Acc: 0.7930 | Val Acc: 0.3112


In [ ]:
results = evaluate_model_multiclass_ohe(
    model=fwm_cnn_model,
    X_test=X_test_fwm,
    y_test=y_test_oh,   # One-Hot
    batch_size=512
)


Accuracy : 0.3118
F1 macro : 0.3126
F1 weighted : 0.2271

Confusion Matrix:
[[23  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 13  4]
 [ 0  0  0 ...  0  5 11]]

Classification Report:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0     0.7931    0.7931    0.7931        29
           1     0.0000    0.0000    0.0000         5
           2     0.0000    0.0000    0.0000      3221
           3     0.9980    0.9713    0.9845       523
           4     1.0000    0.6489    0.7871        94
           5     0.0000    0.0000    0.0000       589
           6     0.9174    0.4411    0.5957       730
           7     0.9818    0.0174    0.0342    117859
           8     0.0645    0.0000    0.0000    105833
           9     0.5184    0.0298    0.0563    106185
          10     0.1793    0.7308    0.2880     98030
          11     0.9837    0.4778    0.6432     11610
          12     0.0000    0.0000    0.0000         1
          13     1.0000    0.7500    0.8571         4
          14     0.3871    0.8494    0.5318    116419
          15     0.2190    0.3510    0.2697    146003
          16     0.8931    0.0011    0.0021    109427
          17     0.9541    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Chemin complet dans Google Drive
save_path = "/content/drive/MyDrive/projet_advanced/fwm_cnn_model.pth"

# Sauvegarde du modèle (state_dict recommandé)
torch.save(fwm_cnn_model.state_dict(), save_path)
print(f"Modèle sauvegardé dans : {save_path}")


Modèle sauvegardé dans : /content/drive/MyDrive/projet_advanced/fwm_cnn_model.pth


### **SWCC-CNN**

In [ ]:
model = SimpleCNN_OneHot(input_channels=1, num_classes=32)
swcc_cnn_model = train_cnn_multiclass_ohe(
    model,
    X_train=X_train_swcc,
    y_train=y_train_oh,   # (N, 8) float
    X_val=X_val_swcc,
    y_val=y_val_oh,       # (N, 8) float
    epochs=20,
    batch_size=256,
    lr=0.001
)

Epoch 1/20 | Train Acc: 0.2390 | Val Acc: 0.1608
Epoch 2/20 | Train Acc: 0.3644 | Val Acc: 0.1523
Epoch 3/20 | Train Acc: 0.4069 | Val Acc: 0.1504
Epoch 4/20 | Train Acc: 0.4316 | Val Acc: 0.2127
Epoch 5/20 | Train Acc: 0.4490 | Val Acc: 0.2061
Epoch 6/20 | Train Acc: 0.4609 | Val Acc: 0.2006
Epoch 7/20 | Train Acc: 0.4698 | Val Acc: 0.1628
Epoch 8/20 | Train Acc: 0.4786 | Val Acc: 0.2442
Epoch 9/20 | Train Acc: 0.4856 | Val Acc: 0.1435
Epoch 10/20 | Train Acc: 0.4923 | Val Acc: 0.2372
Epoch 11/20 | Train Acc: 0.4975 | Val Acc: 0.1704
Epoch 12/20 | Train Acc: 0.5030 | Val Acc: 0.1480
Epoch 13/20 | Train Acc: 0.5071 | Val Acc: 0.2071
Epoch 14/20 | Train Acc: 0.5111 | Val Acc: 0.2491
Epoch 15/20 | Train Acc: 0.5155 | Val Acc: 0.2315
Epoch 16/20 | Train Acc: 0.5184 | Val Acc: 0.2032
Epoch 17/20 | Train Acc: 0.5216 | Val Acc: 0.1994
Epoch 18/20 | Train Acc: 0.5247 | Val Acc: 0.2882
Epoch 19/20 | Train Acc: 0.5281 | Val Acc: 0.2202
Epoch 20/20 | Train Acc: 0.5302 | Val Acc: 0.2147


In [ ]:
results = evaluate_model_multiclass_ohe(
    model=swcc_cnn_model,
    X_test=X_test_swcc,
    y_test=y_test_oh,   # One-Hot
    batch_size=512
)

Accuracy : 0.2153
F1 macro : 0.2594
F1 weighted : 0.221

Confusion Matrix:
[[  21    1    0 ...    0    0    0]
 [   0    5    0 ...    0    0    0]
 [   0    0 3176 ...    0    0    0]
 ...
 [   0    5    0 ...   18    0    0]
 [   0   11    1 ...    0    4   19]
 [   0    3    0 ...    0    0   16]]

Classification Report:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0     0.3182    0.7241    0.4421        29
           1     0.1190    1.0000    0.2128         5
           2     0.8939    0.9860    0.9377      3221
           3     0.5622    0.9847    0.7158       523
           4     0.0263    0.9787    0.0512        94
           5     0.0159    0.8642    0.0311       589
           6     0.0581    0.4493    0.1030       730
           7     0.2412    0.3613    0.2893    117859
           8     0.3288    0.0813    0.1304    105833
           9     0.1762    0.2762    0.2151    106185
          10     0.3396    0.0412    0.0734     98030
          11     0.9015    0.5051    0.6474     11610
          12     0.0000    0.0000    0.0000         1
          13     0.6667    1.0000    0.8000         4
          14     0.2343    0.3861    0.2916    116419
          15     0.5735    0.0334    0.0631    146003
          16     0.2640    0.0009    0.0018    109427
          17     0.9430    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Chemin complet dans Google Drive
save_path = "/content/drive/MyDrive/projet_advanced/swcc_cnn_model.pth"

# Sauvegarde du modèle (state_dict recommandé)
torch.save(swcc_cnn_model.state_dict(), save_path)
print(f"Modèle sauvegardé dans : {save_path}")


In [ ]:
del fwm_cnn_model,fwm_weights,X_test_fwm,X_train_fwm,X_val_fwm